In [1]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns
# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import Dropout
from numpy import asarray



In [2]:
df = pd.read_csv('data/all_states_nn2_balanced.csv')
df.columns

Index(['doctor_altruism', 'doctor_likes_soldier', 'doctor_likes_actor',
       'doctor_has_food', 'doctor_has_meds', 'soldier_altruism',
       'soldier_likes_doctor', 'soldier_likes_actor', 'soldier_has_food',
       'soldier_has_meds', 'actor_altruism', 'actor_likes_doctor',
       'actor_likes_soldier', 'actor_has_food', 'actor_has_meds',
       'food_quantity', 'meds_quantity', 'active', 'results',
       'attack(doctor, actor)', 'attack(doctor, soldier)',
       'prepare_meeting(doctor, actor)', 'prepare_trap(doctor, soldier)',
       'prepare_meeting(doctor, soldier)', 'prepare_trap(doctor, actor)',
       'chosen_action'],
      dtype='object')

In [3]:
# get the dataset
def get_dataset():
	df = pd.read_csv('data/all_states_nn2_balanced.csv')
	#df.drop(['Unnamed: 0','Unnamed: 0.1','results'],axis=1,inplace=True)
	df.drop(['results'],axis=1,inplace=True)
 # Categorical boolean mask
 
	y= df[['chosen_action']]
	
	X = df.drop(['chosen_action'], axis=1)
	print(X.columns)
	print(y.columns)
	categorical_feature_mask = X.dtypes==object
	# filter categorical columns using mask and turn it into a list
	categorical_cols = X.columns[categorical_feature_mask].tolist()

	for col in categorical_cols:
		one_hot = pd.get_dummies(X[col])
		X = X.drop(col,axis = 1)
	# Join the encoded df
		X = X.join(one_hot)
	
	sc = StandardScaler()
	#X = pd.DataFrame(sc.fit_transform(X))
	y_cat = to_categorical(y)
	return X, y_cat

In [4]:
# define dataset
X, y = get_dataset()
# summarize dataset shape
print(X.shape, y.shape)
# summarize first few examples
X.head()
y[0]

Index(['doctor_altruism', 'doctor_likes_soldier', 'doctor_likes_actor',
       'doctor_has_food', 'doctor_has_meds', 'soldier_altruism',
       'soldier_likes_doctor', 'soldier_likes_actor', 'soldier_has_food',
       'soldier_has_meds', 'actor_altruism', 'actor_likes_doctor',
       'actor_likes_soldier', 'actor_has_food', 'actor_has_meds',
       'food_quantity', 'meds_quantity', 'active', 'attack(doctor, actor)',
       'attack(doctor, soldier)', 'prepare_meeting(doctor, actor)',
       'prepare_trap(doctor, soldier)', 'prepare_meeting(doctor, soldier)',
       'prepare_trap(doctor, actor)'],
      dtype='object')
Index(['chosen_action'], dtype='object')
(4511, 26) (4511, 4)


array([0., 0., 0., 1.], dtype=float32)

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X.values, y, test_size=0.2)

In [6]:
model = Sequential()
model.add(Dense(60, input_shape = (26,), activation = "relu"))
model.add(Dense(15, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(4, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                1620      
                                                                 
 dense_1 (Dense)             (None, 15)                915       
                                                                 
 dropout (Dropout)           (None, 15)                0         
                                                                 
 dense_2 (Dense)             (None, 4)                 64        
                                                                 
Total params: 2,599
Trainable params: 2,599
Non-trainable params: 0
_________________________________________________________________


C:\Users\kacpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [7]:
model.fit(x_train, y_train, verbose=1, epochs=10)

Epoch 1/10
113/113 [==============================] - 1s 1ms/step - loss: 0.1081 - accuracy: 0.9820
Epoch 2/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0555 - accuracy: 0.9911
Epoch 3/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0478 - accuracy: 0.9911
Epoch 4/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0425 - accuracy: 0.9911
Epoch 5/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0409 - accuracy: 0.9911
Epoch 6/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0375 - accuracy: 0.9911
Epoch 7/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0318 - accuracy: 0.9911
Epoch 8/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0328 - accuracy: 0.9909
Epoch 9/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0292 - accuracy: 0.9914
Epoch 10/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0264 - accuracy: 0.9917

In [8]:


predict_x=model.predict(x_test) 
y_pred_class=np.argmax(predict_x,axis=1)
from sklearn.metrics import confusion_matrix
y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)
confusion_matrix(y_test_class, y_pred_class)

29/29 [==============================] - 0s 964us/step


array([[  0,   0,   0,   4],
       [  0,   0,   0,   9],
       [  0,   0,   0,   1],
       [  0,   0,   0, 889]], dtype=int64)

In [60]:
from sklearn.metrics import classification_report
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         2
           3       0.99      1.00      0.99       895

    accuracy                           0.99       903
   macro avg       0.25      0.25      0.25       903
weighted avg       0.98      0.99      0.99       903



C:\Users\kacpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kacpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kacpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344

In [61]:
np.set_printoptions(suppress=True)
 
world_state = [-2, #'doctor_altruism'
        2, #'doctor_likes_soldier'
        2, #'doctor_likes_actor'
        0, #'doctor_has_food'
        0, #'doctor_has_meds'
        0, #soldier_altruism
        2, #'soldier_likes_doctor'
        0, #'soldier_likes_actor'
        0, #'soldier_has_food'
        0, # 'soldier_has_meds'
        1, # 'actor_altruism'
        0, #'actor_likes_doctor'
        -1, # 'actor_likes_soldier'
        0, #'actor_has_food'
        0, #'actor_has_meds'
        1, #'food_quantity'
        1, #'meds_quantity'
        0 #'active'
        ]
predicted_actions=[
    
1,#'attack(doctor, actor)'
0,#'attack(doctor, soldier)'
1,#'prepare_meeting(doctor, actor)'
1,#'prepare_trap(doctor, soldier)'
1,#'prepare_meeting(doctor, soldier)'
0#'prepare_trap(doctor, actor)'

]
world_state.extend(predicted_actions)
print(world_state)

yhat = model.predict(asarray([world_state]))
print(np.argmax(yhat, axis=-1))
print('Predicted: %s' % yhat[0])

[-2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 1, 0, -1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0]


ValueError: in user code:

    File "C:\Users\kacpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\kacpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\kacpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\kacpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 2079, in pr